In [ ]:
!pip install gradio

In [ ]:
!pip install ultralytics

gradio code

In [ ]:
import gradio as gr
import numpy as np
import cv2
from PIL import Image
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle

# Load UNet Semantic Segmentation Model
unet_model = load_model("unet.h5", compile=False)

# Load YOLOv8m Object Detection Model
yolo_model = YOLO("yolov8m.pt")

# Color mapping for segmentation
COLOR_MAP = {
    0: [0, 0, 0], 1: [128, 0, 0], 2: [0, 128, 0], 3: [0, 0, 128],
    4: [128, 128, 0], 5: [128, 0, 128], 6: [0, 128, 128], 7: [255, 255, 0],
    8: [0, 255, 0], 9: [255, 0, 0], 10: [0, 255, 255], 11: [255, 165, 0], 12: [255, 255, 255]
}

LABEL_COLORS = {
    "car": "black",
    "truck": "yellow",
    "traffic_light": "purple",
    "pedestrian": "blue",
    "bike": "green"
}

# Function to preprocess image for UNet
def preprocess_image(image):
    original_size = image.size  # Store original size (width, height)
    img = np.array(image)
    img = cv2.resize(img, (256, 192))  # Resize to match UNet model input
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img, original_size

# Function to apply color mapping to segmentation mask
def apply_colormap(pred_mask):
    pred_class = np.argmax(pred_mask, axis=-1)
    color_mask = np.zeros((pred_class.shape[0], pred_class.shape[1], 3), dtype=np.uint8)

    for i in range(pred_class.shape[0]):
        for j in range(pred_class.shape[1]):
            class_id = pred_class[i, j]
            color_mask[i, j] = COLOR_MAP.get(class_id, [0, 0, 0])

    return color_mask

# Function for semantic segmentation
def predict_segmentation(image):
    img, original_size = preprocess_image(image)
    pred_mask = unet_model.predict(img)[0]  # Get prediction
    color_mask = apply_colormap(pred_mask)

    # Resize segmentation mask back to original image size
    color_mask_resized = cv2.resize(color_mask, original_size, interpolation=cv2.INTER_NEAREST)

    return Image.fromarray(color_mask_resized)  # Convert to PIL Image

# Function for object detection
def predict_detection(image):
    input_path = "input.jpg"
    output_path = "output.jpg"

    # Save input image
    image.save(input_path)

    # Run YOLOv8m model
    results = yolo_model.predict(input_path, save=False, conf=0.3, iou=0.5)[0]

    # Draw bounding boxes
    fig, ax = plt.subplots(1, figsize=(10, 6))
    ax.imshow(image)

    for box in results.boxes:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        label_idx = int(box.cls[0])
        label_name = yolo_model.names[label_idx]  # Get class name
        color = LABEL_COLORS.get(label_name, "white")  # Default to white if label is unknown

        # Draw bounding box
        rect = Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor=color, facecolor="none")
        ax.add_patch(rect)

        # Add label and confidence score
        label_text = f"{label_name} {conf:.2f}"
        ax.text(x_min, y_min - 5, label_text, color="white", fontsize=8, bbox=dict(facecolor=color, alpha=0.8, edgecolor="none"))

    ax.axis("off")  # Hide axis
    plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
    plt.close()

    return Image.open(output_path)

# Gradio Interface
iface = gr.Interface(
    fn=lambda image: (predict_segmentation(image), predict_detection(image)),
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil", label="Semantic Segmentation"), gr.Image(type="pil", label="Object Detection")],
    title="Image Analysis: Semantic Segmentation & Object Detection",
    description="Upload an image to perform both semantic segmentation (using UNet) and object detection (using YOLOv8m)."
)

# Launch Gradio App
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ce33702f34fa015637.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')